# Updating `is_commuting` function to accept opmath instances

Currently, the `is_commuting` function does not accept the following operators: `prod`, `sprod` and `sum`. 

The goal is to remove this limitation and accept those operators.

## Implementation Details

In order to make that work:

In pennylane/ops/functions/is_commuting.py:
- Remove `Prod`, `Sum`, `SProd` from the unsupported_operations list.
- Update the logic in is_commuting to support those operations.
- Avoid using matrix multiplication where it is not necessary


In [1]:
import pennylane as qml

from pennylane.pauli.utils import is_pauli_word

In [2]:
import numpy as np
from pennylane.pauli.utils import is_pauli_word, pauli_to_binary, _wire_map_from_pauli_pair


def _pword_is_commuting(pauli_word_1, pauli_word_2, wire_map=None):
    r"""Checks if two Pauli words commute.

    To determine if two Pauli words commute, we can check the value of the
    symplectic inner product of their binary vector representations.
    For two binary vectors representing Pauli words, :math:`p_1 = [x_1, z_1]`
    and :math:`p_2 = [x_2, z_2],` the symplectic inner product is defined as
    :math:`\langle p_1, p_2 \rangle_{symp} = z_1 x_2^T + z_2 x_1^T`. If the symplectic
    product is :math:`0` they commute, while if it is :math:`1`, they don't commute.

    Args:
        pauli_word_1 (Observable): first Pauli word in commutator
        pauli_word_2 (Observable): second Pauli word in commutator
        wire_map (dict[Union[str, int], int]): dictionary containing all wire labels used in
            the Pauli word as keys, and unique integer labels as their values

    Returns:
        bool: returns True if the input Pauli commute, False otherwise

    **Example**

    >>> wire_map = {'a' : 0, 'b' : 1, 'c' : 2}
    >>> pauli_word_1 = qml.X('a') @ qml.Y('b')
    >>> pauli_word_2 = qml.Z('a') @ qml.Z('c')
    >>> is_commuting(pauli_word_1, pauli_word_2, wire_map=wire_map)
    False
    """

    if wire_map is None:
        wire_map = _wire_map_from_pauli_pair(pauli_word_1, pauli_word_2)

    n_qubits = len(wire_map)

    pauli_vec_1 = pauli_to_binary(pauli_word_1, n_qubits=n_qubits, wire_map=wire_map)
    pauli_vec_2 = pauli_to_binary(pauli_word_2, n_qubits=n_qubits, wire_map=wire_map)

    x1, z1 = pauli_vec_1[:n_qubits], pauli_vec_1[n_qubits:]
    x2, z2 = pauli_vec_2[:n_qubits], pauli_vec_2[n_qubits:]

    return (np.dot(z1, x2) + np.dot(z2, x1)) % 2 == 0

In [3]:
_pword_is_commuting(qml.prod(qml.PauliX(0), qml.PauliX(1) @ qml.Identity(2)), qml.PauliX(0))

True

In [4]:
def intersection(wires1, wires2):
    r"""Check if two sets of wires intersect.

    Args:
        wires1 (pennylane.wires.Wires): First set of wires.
        wires2 (pennylane.wires.Wires: Second set of wires.

    Returns:
         bool: True if the two sets of wires are not disjoint and False if disjoint.
    """
    return len(qml.wires.Wires.shared_wires([wires1, wires2])) != 0

@pytest.mark.parametrize(
        "pauli_word_1,pauli_word_2,wire_map,commute_status",
        [
            (qml.Identity(0), qml.PauliZ(0), {0: 0}, True),
            (qml.PauliY(0), qml.PauliZ(0), {0: 0}, False),
            (qml.PauliX(0), qml.PauliX(1), {0: 0, 1: 1}, True),
            (qml.PauliY("x"), qml.PauliX("y"), None, True),
            (
                qml.PauliZ("a") @ qml.PauliY("b") @ qml.PauliZ("d"),
                qml.PauliX("a") @ qml.PauliZ("c") @ qml.PauliY("d"),
                {"a": 0, "b": 1, "c": 2, "d": 3},
                True,
            ),
            (
                qml.PauliX("a") @ qml.PauliY("b") @ qml.PauliZ("d"),
                qml.PauliX("a") @ qml.PauliZ("c") @ qml.PauliY("d"),
                {"a": 0, "b": 1, "c": 2, "d": 3},
                False,
            ),
        ],
    )
    def test_pauli_words(self, pauli_word_1, pauli_word_2, wire_map, commute_status):
        """Test that (non)-commuting Pauli words are correctly identified."""
        do_they_commute = qml.is_commuting(pauli_word_1, pauli_word_2, wire_map=wire_map)
        assert do_they_commute == commute_status

In [5]:
qml.operation.enable_new_opmath()
# qml.operation.disable_new_opmath()
pauli_word_1 = qml.PauliX(0) @ qml.Hadamard(1) @ qml.Identity(2)
pauli_word_2 = qml.PauliX(0) @ qml.PauliY(2)
wire_map = {"a": 0, "b": 0, "c": 2, "d": 3}
assert is_pauli_word(pauli_word_1)
assert is_pauli_word(pauli_word_2)
type(pauli_word_1), type(pauli_word_2)

AssertionError: 

In [59]:
candidate_pauli = qml.s_prod(4, qml.PauliX(0) @ qml.PauliY(1))
is_pauli_word(candidate_pauli)

True

In [7]:
qml.is_commuting(pauli_word_1, pauli_word_2, wire_map=wire_map)

QuantumFunctionError: Prod operations are only supported for Pauli words.

In [8]:
qml.is_commuting(
    qml.sum(qml.PauliY(1), qml.Hadamard(0)), qml.sum(qml.PauliX(0) @ qml.PauliY(1), qml.PauliX(0))
)

Y(1)
Hadamard(wires=[0])


QuantumFunctionError: Sum operations are only supported for Pauli words.

In [9]:
qml.simplify(qml.sum(qml.PauliX(0) @ qml.PauliY(0), qml.Hadamard(0)))

1j * Z(0) + Hadamard(wires=[0])

In [25]:
p1 = (qml.sum(qml.PauliZ("a"), qml.PauliY("b"), qml.PauliZ("d")),)

p1.map_wires

AttributeError: 'tuple' object has no attribute 'map_wires'

In [24]:
p1 = qml.sum(qml.PauliZ(0), qml.PauliY(1), qml.PauliZ(3))
p2 = qml.sum(qml.PauliZ(0), qml.PauliY(1), qml.PauliZ(3))
# {"a": 0, "b": 1, "c": 2, "d": 3},

type(qml.commutator(p1, p2))

qml.is_commuting(p1, p2)

False

In [30]:
p1 = qml.sum(qml.PauliZ("a"), qml.PauliY("b"), qml.PauliZ("d"))
p2 = qml.sum(qml.PauliX("a"), qml.PauliZ("c"), qml.PauliY("d"))
wire_map = {"a": 0, "b": 1, "c": 2, "d": 3}

qml.is_commuting(p1, p2, wire_map)

qml.commutator(p1, p2)

2j * Y('a') + -2j * X('d')

In [57]:
p1 = qml.sum(qml.PauliZ("a"), qml.PauliY("b"), qml.PauliZ("d"))
p2 = qml.sum(qml.PauliX("a"), qml.PauliX("c"), qml.PauliX("d"))
wire_map = {"a": 0, "b": 0, "c": 0, "d": 0}

qml.is_commuting(p1, p2, wire_map)

qml.commutator(p1, p2)



2j * Y('a') + 2j * Y('d')

In [48]:
p1 = qml.sum(qml.PauliZ("a"), qml.PauliY("b"), qml.PauliZ("d"))
p2 = qml.PauliY("a")
wire_map = {"a": 0, "b": 1, "c": 2, "d": 3}

qml.is_commuting(p1, p2, wire_map)

qml.commutator(p1, p2) == qml.s_prod(0, qml.Identity())

qml.commutator(p1, p2)

-2j * X('a')

In [38]:
qml.simplify(qml.s_prod(0, qml.Identity(2)))

0 * I(2)

In [41]:
qml.simplify(qml.s_prod(0, qml.X(0)))

0 * I(0)